In [2]:
import read_files, manage_data
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
path_to_folder = "/eos/user/j/jcapotor/FBGdata/Data/12012023_datosRepetitividadLatiguillo/"
files = [str(int(i)) for i in np.linspace(1,21,21)]

In [4]:
def line(x, A, B):
    return A+B*x

In [ ]:
amp_results = {}
wav_results = {}
fwhm_results = {}
residuals_results = {}
baseline_results = {}
for file in files:
    print("Filling Run" + file)
    peaks = read_files.read_peaks(path_to_folder + "Run"+file+"_peaks.txt")
    peaks_p, peaks_s, peaks_p_full, peaks_s_full = manage_data.process_data(peaks)
    spectrums = read_files.read_spectrums(path_to_folder + "Run"+file+"_spectrum.bin", [0])
    spectrums_p, spectrums_s, spectrums_p_full, spectrums_s_full = manage_data.process_data(spectrums)
    spectrums_p, peaks_p = manage_data.match_dataframes(spectrums_p, peaks_p)

    amp_results["Run"+file] = {}
    wav_results["Run"+file] = {}
    fwhm_results["Run"+file] = {}
    residuals_results["Run"+file] = {}
    baseline_results["Run"+file] = np.mean(spectrums_p["Baseline"])
    for variable in ["Wav1-1", "Wav1-2", "Wav1-3", "Wav1-4"]:
        amp_results["Run"+file]["specs_P_"+variable+"_amp (AU)"] = str(np.round(np.mean(spectrums_p[variable+"_amp"]), 0)) + " +- " + str(np.round(np.std(spectrums_p[variable+"_amp"]),0))
        wav_results["Run"+file]["peaks_P_"+variable] = str(np.round(np.mean(peaks_p[variable]*1e9), 6)) + " +- " + str(np.round(np.std(peaks_p[variable]*1e9), 6)) + " nm"
        wav_results["Run"+file]["specs_P_"+variable] = str(np.round(np.mean(spectrums_p[variable]), 6)) + " +- " + str(np.round(np.std(spectrums_p[variable]), 6)) + " nm"
        fwhm_results["Run"+file]["specs_P_"+variable+"_sigma"] = str(np.round(np.mean(spectrums_p[variable+"_sigma"]), 6)) + " +- " + str(np.round(np.std(spectrums_p[variable+"_sigma"]), 6)) + " nm"
        popt_specs, pcov_specs = curve_fit(line, spectrums_p["Timestamp"], spectrums_p[variable])
        popt_peaks, pcov_peaks = curve_fit(line, peaks_p["Timestamp"], peaks_p[variable]*1e9)
        res_specs = line(spectrums_p["Timestamp"], popt_specs[0], popt_specs[1]) - spectrums_p[variable]
        res_peaks = line(peaks_p["Timestamp"], popt_peaks[0], popt_peaks[1]) - peaks_p[variable]*1e9
        
        residuals_results["Run"+file]["peaks_P"+variable+"_residuals"] = (np.round(np.std(res_peaks*1e6), 1))
        residuals_results["Run"+file]["spectrums_P"+variable+"_residuals"] = (np.round(np.std(res_specs*1e6), 1))

In [10]:
print("Residuals (fm)")
residuals = pd.DataFrame(residuals_results).T
print(residuals)



Residuals (fm)
       peaks_PWav1-1_residuals  spectrums_PWav1-1_residuals  \
Run1                       7.3                          7.8   
Run2                       7.4                          8.8   
Run3                       6.7                          7.2   
Run4                      67.4                          9.7   
Run5                      31.5                         10.9   
Run6                      60.6                         69.7   
Run7                       3.8                          7.9   
Run8                       6.7                         11.0   
Run9                       3.9                          8.8   
Run10                      8.4                         11.2   
Run11                      5.1                         12.0   
Run12                      5.2                         12.2   
Run13                     10.4                         17.4   
Run14                      4.3                          6.6   
Run15                      3.9          

In [11]:
residuals = pd.DataFrame(wav_results).T
print(residuals)
print("Comparison of I4G peak - Spectrums peak (fm)")
comparison = {}
for index, row in residuals.iterrows():
    comparison[index] = {}
    for i in range(1,5):
        peaks_name = "peaks_P_Wav1-"+str(i)
        specs_name = "specs_P_Wav1-"+str(i)
        peaks_value = float(row[peaks_name].split(" ")[0])
        specs_value = float(row[specs_name].split(" ")[0])
        comparison[index]["Wav1-"+str(i)+" (fm)"] = (specs_value - peaks_value)*1e6
print(pd.DataFrame(comparison).T)

                   peaks_P_Wav1-1              specs_P_Wav1-1  \
Run1      1540.069802 +- 9e-06 nm     1540.070584 +- 8e-06 nm   
Run2    1540.069957 +- 1.1e-05 nm   1540.070543 +- 1.8e-05 nm   
Run3    1540.070077 +- 2.4e-05 nm   1540.070428 +- 2.2e-05 nm   
Run4       1540.08638 +- 7e-05 nm   1540.070265 +- 1.9e-05 nm   
Run5      1539.974339 +- 6e-05 nm   1540.070008 +- 2.7e-05 nm   
Run6    1540.069418 +- 7.2e-05 nm   1540.069787 +- 8.7e-05 nm   
Run7      1540.069414 +- 4e-06 nm     1540.069739 +- 8e-06 nm   
Run8      1540.069436 +- 7e-06 nm   1540.069747 +- 1.1e-05 nm   
Run9      1540.069603 +- 8e-06 nm    1540.06992 +- 1.1e-05 nm   
Run10   1540.069444 +- 2.7e-05 nm   1540.069788 +- 3.9e-05 nm   
Run11    1540.06998 +- 3.8e-05 nm   1540.070524 +- 5.9e-05 nm   
Run12   1540.070145 +- 2.9e-05 nm   1540.070757 +- 4.4e-05 nm   
Run13    1540.07031 +- 2.9e-05 nm   1540.070978 +- 4.6e-05 nm   
Run14   1540.070407 +- 1.6e-05 nm   1540.071123 +- 2.3e-05 nm   
Run15   1540.070487 +- 1.

In [12]:
print(pd.DataFrame(amp_results).T)

      specs_P_Wav1-1_amp (AU) specs_P_Wav1-2_amp (AU) specs_P_Wav1-3_amp (AU)  \
Run1            4201.0 +- 1.0           4106.0 +- 3.0           3870.0 +- 2.0   
Run2            4201.0 +- 1.0           4124.0 +- 2.0           3858.0 +- 4.0   
Run3            4197.0 +- 2.0           4144.0 +- 3.0           3844.0 +- 2.0   
Run4            4196.0 +- 2.0           4159.0 +- 4.0           3827.0 +- 3.0   
Run5            4201.0 +- 2.0           4173.0 +- 1.0           3817.0 +- 2.0   
Run6            4188.0 +- 5.0           4149.0 +- 7.0           3794.0 +- 7.0   
Run7            4196.0 +- 2.0           4142.0 +- 3.0           3797.0 +- 3.0   
Run8            4204.0 +- 3.0           4136.0 +- 3.0           3802.0 +- 1.0   
Run9            4214.0 +- 1.0           4122.0 +- 1.0           3808.0 +- 1.0   
Run10           4223.0 +- 2.0           4108.0 +- 3.0           3812.0 +- 2.0   
Run11           4236.0 +- 1.0           4061.0 +- 2.0           3817.0 +- 1.0   
Run12           4237.0 +- 2.

In [13]:
print("Sigma of gaussian fit (pm)")
print(pd.DataFrame(fwhm_results).T)

Sigma of gaussian fit (pm)
          specs_P_Wav1-1_sigma     specs_P_Wav1-2_sigma  \
Run1    0.06343 +- 0.008531 nm  0.083382 +- 0.008582 nm   
Run2   0.068345 +- 0.007391 nm  0.074284 +- 0.004939 nm   
Run3     0.06632 +- 0.00548 nm  0.068273 +- 0.004261 nm   
Run4   0.071787 +- 0.007773 nm  0.065998 +- 0.003716 nm   
Run5   0.067566 +- 0.006989 nm  0.060131 +- 0.002151 nm   
Run6   0.070464 +- 0.007635 nm   0.06124 +- 0.001848 nm   
Run7   0.058822 +- 0.006871 nm   0.061552 +- 0.00283 nm   
Run8   0.057203 +- 0.005989 nm  0.059841 +- 0.002223 nm   
Run9     0.0663 +- 0.004986 nm  0.063541 +- 0.003159 nm   
Run10  0.062669 +- 0.003774 nm  0.065711 +- 0.004531 nm   
Run11  0.058765 +- 0.002808 nm   0.070165 +- 0.00507 nm   
Run12  0.058366 +- 0.003633 nm  0.072134 +- 0.005354 nm   
Run13  0.061165 +- 0.003905 nm  0.070856 +- 0.005939 nm   
Run14   0.06343 +- 0.004737 nm  0.073488 +- 0.007861 nm   
Run15  0.062264 +- 0.003528 nm  0.071622 +- 0.008445 nm   
Run16  0.061121 +- 0.003685 n